### Analyze + Plot Eye Saccades
The goal of this script is to determine wether the animal's saccades
are aligned with the location of the stimulus. In other words, if the
animal is looking at the salient part of the screen. The script:

  - Requires a path to a folder on the lab's server (X:\)
  - Loads right- and left-eye Bonsai CSV outputs, IMU & stimulus TTL files.
  - Computes eye-centric gaze, detects saccades (optionally removing blinks).
  - Creates three kinds of figures and writes them to  <folder_path>/Results/ :
      ─ “ALL”  : quiver of every saccade  + polar & linear angle histograms
      ─ LR/UD  : same layout but restricted to each stimulus direction
  - Global X/Y limits are derived from the full session; every quiver plot
    shares them so comparisons are direct.
  - Figure filenames:  <session>_<Eye>_<condition>.png
      e.g.  Tsh001_2025-06-11T13_02_29_Right_ALL_Interleaved.png


### How saccades are detected in this script

1. **Eye-centred coordinates**  
   Subtract the mid-point of the two eye-corner markers so gaze is relative to the eye, not the camera.

2. **Denoise (`medfilt_vec`)**  
   A 3-point median filter removes single-frame tracking jitter.

3. **Pixel → degree conversion**  
   Divide by the calibration factor ( ≈ 3.76 px = 1 °) to work in visual degrees.

4. **Instantaneous velocity (`frame_velocity`)**  
   Take the frame-to-frame difference in x and y; combine into a scalar speed (deg / frame).

5. **Speed threshold**  
   If speed ≥ `1.5 deg/frame`, mark that frame as a saccade (`saccade_indices`).

6. **Blink removal (`blink_frames`, optional)**  
   When VD-axis files are present, large jumps in eyelid separation flag blinks; those frames are deleted from `saccade_indices`.

The cleaned list **`saccade_indices`** feeds every quiver plot, PCA arrow, and angle histogram in the notebook.




Tested with Python 3.12 (Conda env “EyeHeadCoupling”).

  Author:  <Ratnadeep Pal @SarvestaniLab>   –  Last update: 2025-06-15

## Import the required libraries


In [45]:

import sys
import os
import numpy as np
import pandas as pd
from scipy.fft import fft
from scipy.signal import ShortTimeFFT,butter,hilbert,sosfiltfilt,medfilt
from scipy.signal.windows import gaussian
import scipy.stats as stats
import matplotlib.pyplot as plt
import matplotlib.colors
from matplotlib.ticker import ScalarFormatter
import tkinter as tk
from tkinter import filedialog
from sklearn.decomposition import PCA
import os
from io import StringIO
from matplotlib import gridspec
from pathlib import Path
import matplotlib.lines as mlines
import matplotlib.gridspec as gridspec


%matplotlib qt


## Define parameters

In [46]:
cal = 3.76  # Calibration factor for the pixels to degrees
ttl_freq = 60  # TTL frequency in Hz

# Parameters for nlink and saccade detection
blink_thresh= 10
saccade_thresh= 1
saccade_win=0.5     # Window size for saccade detection in seconds

#folder_path = select_folder() #this won't work if you're running jupyter lab in browser, so hard coding


###################################### Paris

#First day when we started doing interleaved stim.
#folder_path = r"X:\Experimental_Data\EyeHeadCoupling_RatTS_server\TSh01_Paris_server\Tsh001_2025-06-09T13_09_02\\" # interleaved

#Second day where she licked a lot!
#folder_path = r"X:\Experimental_Data\EyeHeadCoupling_RatTS_server\TSh01_Paris_server\Tsh001_2025-06-11T12_50_45\\" #no stim
folder_path = r"X:\Experimental_Data\EyeHeadCoupling_RatTS_server\TSh01_Paris_server\Tsh001_2025-06-11T13_02_29\\" # interleaved
#folder_path = r"X:\Experimental_Data\EyeHeadCoupling_RatTS_server\TSh01_Paris_server\Tsh001_2025-06-11T13_15_39\\" #  interleaved

#Not much licking on this day
#folder_path = r"X:\Experimental_Data\EyeHeadCoupling_RatTS_server\TSh01_Paris_server\Tsh001_2025-06-13T13_07_55\\" #interleaved
#folder_path = r"X:\Experimental_Data\EyeHeadCoupling_RatTS_server\TSh01_Paris_server\Tsh001_2025-06-13T13_25_04\\" #no-stim session

#Motivated on this day, but first day where juice was only given for saccades
#folder_path = r"X:\Experimental_Data\EyeHeadCoupling_RatTS_server\TSh01_Paris_server\Tsh001_2025-06-16T12_13_51\\" #interleaved


######################################## Bayleaf
#folder_path = r"X:\Experimental_Data\EyeHeadCoupling_RatTS_server\Rat22_Bayleaf_server\Rat022_2025-06-10T16_23_02\\"   #no stim
#folder_path = r"X:\Experimental_Data\EyeHeadCoupling_RatTS_server\Rat22_Bayleaf_server\Rat022_2025-06-10T16_10_21\\"   #no stim


results_dir = Path(folder_path) / "Results\\"
results_dir.mkdir(exist_ok=True)


## Define functions


In [ ]:
# Prompt the user to select a folder
def select_folder():
    root = tk.Tk()
    root.withdraw()  # Hide the main window
    directory = filedialog.askdirectory()  # Open the file selection dialog
    return directory

# Prompt the user to open a file 
def select_file():
    root = tk.Tk()
    root.withdraw()  # Hide the main window
    file_path = filedialog.askopenfilename()  # Open the file selection dialog
    return file_path

def choose_option(option1, option2, option3, option4):
    result = {}

    def select(choice):
        result['value'] = choice
        root.destroy()

    root = tk.Tk()
    root.title("Choose the type of visual stim")

    tk.Label(root, text="Please choose the type of visual stim:").pack(pady=10)
    tk.Button(root, text=option1, width=12, command=lambda: select(option1)).pack(side='left', padx=10, pady=10)
    tk.Button(root, text=option2, width=12, command=lambda: select(option2)).pack(side='left', padx=10, pady=10)
    tk.Button(root, text=option3, width=12, command=lambda: select(option3)).pack(side='left', padx=10, pady=10)
    tk.Button(root, text=option4, width=12, command=lambda: select(option4)).pack(side='left', padx=10, pady=10)

    # Manual event loop, blocks until window is destroyed
    while not result.get('value'):
        root.update()

    return result['value']

# Prompt the user to choose the type of visual stim
stim_type = choose_option("None","LR","UD","Interleaved")


# Function to remove parentheses characters from a line
def remove_parentheses_chars(line):
    # Remove only '(' and ')' characters
    return line.replace('(', '').replace(')', '')
def clean_csv(filename):
    with open(filename, 'r') as f:
        lines = [remove_parentheses_chars(line) for line in f]
    # Join lines and create a file-like object
        cleaned = StringIO(''.join(lines))
        return cleaned

# Butterworth filter to remove high frequency noise
def butter_noncausal(signal, fs, cutoff_freq=1, order=4):
    sos = butter(order, cutoff_freq/(fs/2), btype='low', output='sos')  # 50 Hz cutoff frequency
    return sosfiltfilt(sos, signal)   

def interpolate_nans(arr):
    nans = np.isnan(arr)
    x = np.arange(len(arr))
    arr[nans] = np.interp(x[nans], x[~nans], arr[~nans])
    return arr

def rotation_matrix(angle_rad):
    return np.array([[np.cos(angle_rad), -np.sin(angle_rad)],
                     [np.sin(angle_rad), np.cos(angle_rad)]])


pca = PCA(n_components=2)

def vector_to_rgb(angle, absolute): ##Got it from https://stackoverflow.com/questions/19576495/color-matplotlib-quiver-field-according-to-magnitude-and-direction
    global max_abs

    # normalize angle
    angle = angle % (2 * np.pi)
    if angle < 0:
        angle += 2 * np.pi

    # return matplotlib.colors.hsv_to_rgb((angle / 2 / np.pi, 
    #                                      absolute / max_abs, 
    #                                      absolute / max_abs))
    return matplotlib.colors.hsv_to_rgb((angle / 2 / np.pi, 
                                         1, 
                                         1))
def plot_angle_distribution(angle, ax_polar, num_bins=18):
    """
    Plots a normalized polar histogram of angles.

    Parameters:
        angle (np.ndarray): array of saccade angles in radians
        ax_polar (matplotlib.axes._subplots.PolarAxesSubplot): the polar subplot to draw on
        num_bins (int): number of histogram bins
    """
    angle_2pi = np.where(angle < 0, angle + 2 * np.pi, angle)
    counts, bin_edges = np.histogram(angle_2pi, bins=num_bins, range=(0, 2 * np.pi))
    counts = counts / np.size(angle_2pi)  # Normalize
    width = np.diff(bin_edges)

    bars = ax_polar.bar(bin_edges[:-1], counts, width=width, align='edge', color='b', alpha=0.5, edgecolor='k')
    ax_polar.set_title("Normalized angle distribution")
    ax_polar.set_yticklabels([])

def plot_linear_histogram(angles, ax, num_bins=18):
    ang_deg = np.degrees(angles)
    ang_deg = np.mod(ang_deg, 360)
    counts, bins = np.histogram(ang_deg, bins=num_bins, range=(0, 360))
    counts = counts / ang_deg.size
    ax.bar(bins[:-1], counts, width=np.diff(bins), color="b", alpha=0.5, edgecolor="k")
    ax.set_xlabel("Angle (deg)")
    ax.set_ylabel("Normalised count")
    ax.set_title("Linear angle histogram")    

def analyze_eye_saccades(
    marker1_x, marker1_y, marker2_x, marker2_y,
    gaze_x, gaze_y,
    eye_frames,
    calibration_factor,
    go_frame, 
    blink_velocity_threshold, 
    saccade_threshold,
    go_dir_x = None,
    go_dir_y = None,
    stim_type = "None",
    blink_detection=0,
    vd_axis_lx=None, vd_axis_ly=None, vd_axis_rx=None, vd_axis_ry=None,
):
    
    # 1. eye-centred coordinates  →  degrees
    eye_origin = np.column_stack((
        (marker1_x + marker2_x) / 2,
        (marker1_y + marker2_y) / 2
    ))
    eye_angle = np.arctan2(marker2_y - marker1_y, marker2_x - marker1_x)

    eye_camera = np.column_stack((gaze_x - eye_origin[:, 0], gaze_y - eye_origin[:, 1]))
    eye_camera[:, 0] = medfilt(eye_camera[:, 0], kernel_size=3)
    eye_camera[:, 1] = medfilt(eye_camera[:, 1], kernel_size=3)
    eye_camera = eye_camera / calibration_factor

    # 2. instantaneous velocity  →  speed
    eye_vel = np.zeros_like(eye_camera)
    eye_vel[:, 0] = np.ediff1d(eye_camera[:, 0], to_begin=0)
    eye_vel[:, 1] = np.ediff1d(eye_camera[:, 1], to_begin=0)
    speed   = np.linalg.norm(eye_vel, axis=1)

    mask = speed >= saccade_threshold
    saccade_indices = np.where(mask)[0] # ← row indices (0…7158)
    saccade_frames = eye_frames[saccade_indices] # ← absolute Bonsai frames

    fig, ax = plt.subplots(figsize=(12, 4))

    frames = np.arange(len(speed))

    # 1) speed trace
    ax.plot(frames, speed, linewidth=0.8, label='Speed (°/frame)')

    # 2) highlight saccade frames
    ax.scatter(saccade_indices, speed[saccade_indices],
            color='tab:red', s=12, label='Saccade idx')

    # 3) threshold line
    ax.axhline(saccade_threshold, color='tab:orange',
            linestyle='--', label=f'Threshold = {saccade_threshold}')

    ax.set_xlabel('Frame number')
    ax.set_ylabel('Speed (° / frame)')
    ax.set_title('Instantaneous eye speed with detected saccade frames')
    ax.legend()
    ax.grid(alpha=.3)
    plt.tight_layout()
    plt.show()


    # 3. optional blink removal
    if blink_detection:
        vd_axis_left = np.vstack([vd_axis_lx, vd_axis_ly]).T
        vd_axis_right = np.vstack([vd_axis_rx, vd_axis_ry]).T
        vd_axis_d = np.linalg.norm(vd_axis_right - vd_axis_left, axis=1)
        vd_axis_vel = np.gradient(vd_axis_d)
        blink_indices = np.where(np.abs(vd_axis_vel) > blink_velocity_threshold)
        saccade_indices = saccade_indices[~np.isin(saccade_indices, blink_indices)]

    # 4. build stimulus-direction sets
    direction_sets = {}
    if stim_type == "LR":
        direction_sets = {"Left": go_dir_x < 0,
                        "Right": go_dir_x > 0}
    elif stim_type == "UD":
        direction_sets = {"Down": go_dir_y < 0,
                        "Up":   go_dir_y > 0}
    elif stim_type == "Interleaved":
        direction_sets = {
            "Left":  go_dir_x < 0,
            "Right": go_dir_x > 0,
            "Down":  go_dir_y < 0,
            "Up":    go_dir_y > 0
        }
    else:                   # "None"
        direction_sets = {"All": np.full(len(go_frame), True)}

    stim_frames = {lab: go_frame[mask] for lab, mask in direction_sets.items()}


    return {
        "eye_camera":       eye_camera,
        "eye_vel":          eye_vel,
        "saccade_indices":  saccade_indices,
        "stim_frames":      stim_frames,
        "saccade_frames":   saccade_frames,
        }



def plot_eye_saccades(
    eye_camera, eye_camera_diff, saccade_indices, saccade_frames,
    stim_frames,                # dict: label → array of frames
    saccade_window,  # seconds before/after each stimulus
    session_path,
    stim_type='None',
    eye_name='Eye',
):

    session_name = os.path.basename(session_path.rstrip("/\\"))

    # ───────── global axis limits (all saccades) ─────────
    x_all = eye_camera[saccade_indices, 0]
    y_all = eye_camera[saccade_indices, 1]
    pad   = 0.10
    rngX  = x_all.max() - x_all.min()
    rngY  = y_all.max() - y_all.min()
    X_LIM = (x_all.min() - pad*rngX, x_all.max() + pad*rngX)
    Y_LIM = (y_all.min() - pad*rngY, y_all.max() + pad*rngY)

    max_abs = np.max(np.abs(eye_camera_diff))
    angle_all = np.arctan2(eye_camera_diff[saccade_indices, 1],
                            eye_camera_diff[saccade_indices, 0])
    n_all = len(saccade_indices)

    # ───────── master figure (ALL saccades) ─────────
    fig = plt.figure(figsize=(10, 6))
    gs  = gridspec.GridSpec(2, 2, width_ratios=[3, 2])
    ax_quiver = fig.add_subplot(gs[:, 0])
    ax_polar  = fig.add_subplot(gs[0, 1], polar=True)
    ax_linear = fig.add_subplot(gs[1, 1])

    ax_quiver.set_xlim(*X_LIM); ax_quiver.set_ylim(*Y_LIM)
    ax_quiver.set_xlabel('X (°)'); ax_quiver.set_ylabel('Y (°)')
    ax_quiver.set_title(
        f"{session_name}\nAll saccades ({n_all}) — {eye_name}  (stim: {stim_type})"
    )

    cols = np.array([vector_to_rgb(a, max_abs) for a in angle_all])
    ax_quiver.quiver(x_all, y_all,
                        eye_camera_diff[saccade_indices, 0],
                        eye_camera_diff[saccade_indices, 1],
                        angles='xy', scale_units='xy', scale=1,
                        color=cols, alpha=.5)

    # PCA arrows (unchanged)
    pca.fit(eye_camera_diff[saccade_indices] /
            np.linalg.norm(eye_camera_diff[saccade_indices], axis=1, keepdims=True))
    for i, (vec, var) in enumerate(zip(pca.components_, pca.explained_variance_ratio_)):
        ax_quiver.arrow(np.mean(x_all), np.mean(y_all),
                        *(vec * 10 * np.sqrt(var)),
                        color=['k', 'b'][i], width=0.1,
                        label=f'PC{i+1} ({var:.2f} var)')
    ax_quiver.legend()

    plot_angle_distribution(angle_all, ax_polar)
    plot_linear_histogram(angle_all, ax_linear)

    # save master figure
    all_fname = f"{session_name}_{eye_name}_ALL_{stim_type}.png"
    fig.savefig(results_dir / all_fname, dpi=300, bbox_inches='tight')


    # Determine the overall frame range [0, last_frame]
    last_frame = int(saccade_frames.max())
    clipped_any = False
    plot_window = np.arange(0,saccade_window,1)

    # ───────── one figure per stimulus label (skip "All") ─────────
    for label, frames in stim_frames.items():
        if label == "All":
            continue

        # gather saccades within ±plot_window around each stim

        idx_buf = []  # buffer to collect saccade indices for this label

        for f in frames:
            lower_bound = max(f + plot_window[0], 0)
            upper_bound = min(f + plot_window[-1], saccade_frames.max())

            for sf, idx in zip(saccade_frames, saccade_indices):
                if lower_bound <= sf <= upper_bound:
                    idx_buf.append(idx)


        idx_use = np.array(idx_buf, dtype=int)
        if idx_use.size == 0:
            continue

        ang = np.arctan2(eye_camera_diff[idx_use, 1],
                            eye_camera_diff[idx_use, 0])
        n_cond = len(idx_use)

        fig = plt.figure(figsize=(6, 3))
        gs  = gridspec.GridSpec(2, 2, width_ratios=[3, 2])
        ax_q = fig.add_subplot(gs[:, 0])
        ax_p = fig.add_subplot(gs[0, 1], polar=True)
        ax_l = fig.add_subplot(gs[1, 1])

        ax_q.set_xlim(*X_LIM); ax_q.set_ylim(*Y_LIM)
        ax_q.set_xlabel('X (°)'); ax_q.set_ylabel('Y (°)')
        ax_q.set_title(f"{session_name}\n{eye_name} — {label} (n={n_cond})")

        cols = np.array([vector_to_rgb(a, max_abs) for a in ang])
        ax_q.quiver(eye_camera[idx_use, 0], eye_camera[idx_use, 1],
                    eye_camera_diff[idx_use, 0], eye_camera_diff[idx_use, 1],
                    angles='xy', scale_units='xy', scale=1,
                    color=cols, alpha=.5)

        plot_angle_distribution(ang, ax_p)
        plot_linear_histogram(ang, ax_l)

        fig.tight_layout()
        fname = f"{session_name}_{eye_name}_{label.replace('/','-')}.png"
        fig.savefig(results_dir / fname, dpi=300, bbox_inches='tight')



















def plot_saccade_probability(
    go_frame,            # array of stimulus frames
    go_dir_x=None,       # LR direction code  (same len as go_frame)
    go_dir_y=None,       # UD direction code  (same len as go_frame)
    saccade_frames=None, # frame IDs of every saccade
    ttl_freq     = 60,   # Hz
    t_window_s   = 0.5,  # +/- seconds around stimulus
    bin_ms       = 50    # bin width (ms)
):
    """Line plot of P(saccade) vs time, with 5 curves:
       All, Left, Right, Down, Up."""

    assert saccade_frames is not None, "Need array of saccade frame indices"

    # ---------- binning setup -----------------------------------------
    frame_win  = int(t_window_s * ttl_freq)
    bin_frames = int((bin_ms / 1000) * ttl_freq)
    bins       = np.arange(-frame_win, frame_win + bin_frames, bin_frames)
    t_sec      = (bins[:-1] + bin_frames/2) / ttl_freq   # bin centres in sec

    def peristim_prob(stim_frames):
        """Histogram of saccade counts per stimulus."""
        if len(stim_frames) == 0:
            return np.full(len(t_sec), np.nan)
        rel = []
        for f0 in stim_frames:
            idx = saccade_frames[
                (saccade_frames >= f0 - frame_win) &
                (saccade_frames <= f0 + frame_win)
            ]
            rel.extend(idx - f0)
        counts, _ = np.histogram(rel, bins=bins)
        return counts / len(stim_frames)   # probability per bin

    # ---------- classify stimuli --------------------------------------
    eps = 1e-6
    gx  = go_dir_x if go_dir_x is not None else np.zeros_like(go_frame)
    gy  = go_dir_y if go_dir_y is not None else np.zeros_like(go_frame)

    mask_left  = (gx < -eps) & (np.abs(gy) < eps)
    mask_right = (gx >  eps) & (np.abs(gy) < eps)
    mask_down  = (gy < -eps)
    mask_up    = (gy >  eps)

    stims = {
        'All'  : (np.ones_like(go_frame,  dtype=bool), 'k',    2.0),
        'Left' : (mask_left,  'green', 1.4),
        'Right': (mask_right, 'pink',  1.4),
        'Down' : (mask_down,  'blue',  1.4),
        'Up'   : (mask_up,    'red',   1.4)
    }

    # ---------- plot --------------------------------------------------
    fig, ax = plt.subplots(figsize=(6,4))

    for lbl, (m, col, lw) in stims.items():
        stim_frames = go_frame[m]
        p = peristim_prob(stim_frames)
        if np.all(np.isnan(p)):            # skip if no stimuli of that type
            continue
        ax.plot(t_sec, p, color=col, lw=lw, label=f"{lbl} (n={len(stim_frames)})")

    ax.axvline(0, color='k', ls='--', alpha=.6)
    ax.set_xlabel("Time relative to stimulus onset (s)")
    ax.set_ylabel(f"P(saccade) per {bin_ms} ms bin")
    ax.set_title(f"Saccade probability (±{t_window_s}s window)")
    ax.set_xlim(-t_window_s, t_window_s)
    ax.grid(alpha=.3)
    ax.legend()
    plt.tight_layout()
    return fig

## Extract relevant files from filepath

In [48]:
#initialize variables to store file paths
IMU_file = None
camera_file = None
go_file = None
ellipse_center_XY_R_file = None
origin_of_eye_coordinate_R_file = None
vdaxis_R_file = None
ellipse_center_XY_L_file = None
origin_of_eye_coordinate_L_file = None
vdaxis_L_file = None
blink_detection_r = 0
blink_detection_l = 0

print(f"Scanning folder: {folder_path}")
print(f"Found {len(os.listdir(folder_path))} files")

# Scan the folder for specific files
for f in os.listdir(folder_path):
    f_lower = f.lower()
    full_path = os.path.join(folder_path, f)

    if 'imu' in f_lower:
        IMU_file = full_path
    elif 'camera' in f_lower:
        camera_file = full_path
    elif 'go' in f_lower:
        go_file = full_path
    elif 'ellipse_center_xy_r' in f_lower:
        ellipse_center_XY_R_file = full_path
    elif 'origin_of_eyecoordinate_r' in f_lower:
        origin_of_eye_coordinate_R_file = full_path
    elif 'vdaxis_r' in f_lower:
        vdaxis_R_file = full_path
        blink_detection_r = 1
    elif 'ellipse_center_xy_l' in f_lower:
        ellipse_center_XY_L_file = full_path
    elif 'origin_of_eyecoordinate_l' in f_lower:
        origin_of_eye_coordinate_L_file = full_path
    elif 'vdaxis_l' in f_lower:
        vdaxis_L_file = full_path
        blink_detection_l = 1

# Check if all required files are found
if IMU_file == None and camera_file == None:
    raise ValueError('IMU and Camera files not found in the selected folder.')
if IMU_file == None:
    raise ValueError('IMU file not found in the selected folder.')
if camera_file == None:
    raise ValueError('Camera file not found in the selected folder.')
if go_file == None:
    raise ValueError('go file not found in the selected folder.')
if ellipse_center_XY_R_file == None:
    raise ValueError('ellipse_center_XY_R file not found in the selected folder.')
if origin_of_eye_coordinate_R_file == None: 
    raise ValueError('origin_of_eye_coordinate_R file not found in the selected folder.')
if ellipse_center_XY_L_file == None:
    raise ValueError('ellipse_center_XY_L file not found in the selected folder.')  
if origin_of_eye_coordinate_L_file == None:
    raise ValueError('origin_of_eye_coordinate_L file not found in the selected folder.')
if vdaxis_R_file == None:
    print("Warning: Right VD Axis file is not present. Saccade detection might be bad especially for rats!!:")
if vdaxis_L_file == None:
    print("Warning: Left VD Axis file is not present. Saccade detection might be bad especially for rats!!")



Scanning folder: X:\Experimental_Data\EyeHeadCoupling_RatTS_server\TSh01_Paris_server\Tsh001_2025-06-11T13_02_29\\
Found 24 files


## Extract stim and eye position data (Right, Left only if it exists)

In [49]:
## Read the camera data and map between camera TTL (for saccades) and Bonsai TTLs (for frames)
camera_data = np.genfromtxt(camera_file, delimiter=',', skip_header=1, dtype=np.float64)
[bonsai_frame, bonsai_time] = camera_data[:, 0], camera_data[:, 1]
bonsai_frame = bonsai_frame.astype(int)  # Convert bonsai_frame to integer type


### Read the go file for the start of stim in the trial 
new_go_data_format=0
go_data = np.genfromtxt(clean_csv(go_file), delimiter=',', skip_header=1, dtype=np.float64)
if go_data.shape[1]>3:
    new_go_data_format = 1
    [go_frame, go_time, go_direction_x,go_direction_y] = go_data[:, 0], go_data[:, 1], go_data[:, 2], go_data[:,3]
else:
    [go_frame, go_time, go_direction] = go_data[:, 0], go_data[:, 1], go_data[:, 2]
go_frame = go_frame.astype(int)  # Convert go_frame to integer type

### Read the ellipse center XY R file for the right eye 
ellipse_center_XY_R_data = np.genfromtxt(clean_csv(ellipse_center_XY_R_file), delimiter=',', skip_header=1, dtype=np.float64)
[eye_frame_r,eye_timestamp_r,eye_rx,eye_ry] = ellipse_center_XY_R_data[:, 0], ellipse_center_XY_R_data[:, 1], ellipse_center_XY_R_data[:, 2], ellipse_center_XY_R_data[:, 3]
eye_frame_r = eye_frame_r.astype(int)  # Convert eye_frame_r to integer type
eye_rx = interpolate_nans(eye_rx)  # Interpolate NaN values in eye_rx
eye_ry = interpolate_nans(eye_ry)  # Interpolate NaN values in eye_ry

### Read the origin of eye coordinate R file for the right eye 
origin_of_eye_coordinate_R_data = np.genfromtxt(clean_csv(origin_of_eye_coordinate_R_file), delimiter=',', skip_header=1, dtype=np.float64)
[origin_frame_r,o_ts,l_rx,l_ry,r_rx,r_ry] = origin_of_eye_coordinate_R_data[:, 0], origin_of_eye_coordinate_R_data[:, 1], origin_of_eye_coordinate_R_data[:, 2], origin_of_eye_coordinate_R_data[:, 3], origin_of_eye_coordinate_R_data[:, 4], origin_of_eye_coordinate_R_data[:, 5]
origin_frame_r = origin_frame_r.astype(int)  # Convert origin_frame_r to integer type
l_rx = interpolate_nans(l_rx)  # Interpolate NaN values in l_rx
r_rx = interpolate_nans(r_rx)  # Interpolate NaN values in r_rx 
l_ry = interpolate_nans(l_ry)  # Interpolate NaN values in l_ry
r_ry = interpolate_nans(r_ry)  # Interpolate NaN values in r_ry

## Read the vertical (VD) axis data for right eye - this is used for blink detection
if (blink_detection_r == 1):
    vdaxis_R_data = np.genfromtxt(clean_csv(vdaxis_R_file),delimiter=',',skip_header=1,dtype=np.float64)
    [vd_frame_r,vd_r_ts,vd_r_lx,vd_r_ly,vd_r_rx,vd_r_ry] = vdaxis_R_data[:,0],vdaxis_R_data[:,1],vdaxis_R_data[:,2],vdaxis_R_data[:,3],vdaxis_R_data[:,4],vdaxis_R_data[:,5]
    vd_frame_r = vd_frame_r.astype(int)
    # Interpolate NaN values
    vd_r_lx = interpolate_nans(vd_r_lx)
    vd_r_ly = interpolate_nans(vd_r_ly)
    vd_r_rx = interpolate_nans(vd_r_rx)
    vd_r_ry = interpolate_nans(vd_r_ry)

# Check if the left eye data files exist and read them if they do
ellipse_center_XY_L_file = ellipse_center_XY_L_file if 'ellipse_center_xy_l' in locals() else None
origin_of_eye_coordinate_L_file = origin_of_eye_coordinate_L_file if 'origin_of_eyecoordinate_l' in locals() else None
vdaxis_L_file = vdaxis_L_file if 'vdaxis_l' in locals() else None

# Initialize variables for left eye data
eye_frame_l = None
eye_timestamp_l = None
eye_lx = None
eye_ly = None
origin_frame_l = None
l_lx = None
r_lx = None
l_ly = None

r_ly = None
vd_frame_l = None
vd_l_ts = None
vd_l_lx = None
vd_l_ly = None
vd_l_rx = None
vd_l_ry = None
# Attempt to read the left eye data files if they exist
if ellipse_center_XY_L_file and origin_of_eye_coordinate_L_file:
    print("Reading left eye data files...")
    blink_detection_l = 1  # Set blink detection for left eye to 1 if files are present
    try:
        ### Read the ellipse center XY L file for the left eye using numpy
        ellipse_center_XY_L_data = np.genfromtxt(clean_csv(ellipse_center_XY_L_file), delimiter=',', skip_header=1, dtype=np.float64)
        [eye_frame_l,eye_timestamp_l,eye_lx,eye_ly] = ellipse_center_XY_L_data[:, 0], ellipse_center_XY_L_data[:, 1], ellipse_center_XY_L_data[:, 2], ellipse_center_XY_L_data[:, 3]
        eye_frame_l = eye_frame_l.astype(int)  # Convert eye_frame_l to integer type
        eye_lx = interpolate_nans(eye_lx)  # Interpolate NaN values in eye_lx
        eye_ly = interpolate_nans(eye_ly)  # Interpolate NaN values in eye_ly

        ### Read the origin of eye coordinate L file for the left eye using numpy       
        origin_of_eye_coordinate_L_data = np.genfromtxt(clean_csv(origin_of_eye_coordinate_L_file), delimiter=',', skip_header=1, dtype=np.float64)
        [origin_frame_l,o_ts_l,l_lx,l_ly,r_lx,r_ly] = origin_of_eye_coordinate_L_data[:, 0], origin_of_eye_coordinate_L_data[:, 1], origin_of_eye_coordinate_L_data[:, 2], origin_of_eye_coordinate_L_data[:, 3], origin_of_eye_coordinate_L_data[:, 4], origin_of_eye_coordinate_L_data[:, 5]
        origin_frame_l = origin_frame_l.astype(int)  # Convert origin_frame_l to integer type
        l_lx = interpolate_nans(l_lx)  # Interpolate NaN values in l_lx
        r_lx = interpolate_nans(r_lx)  # Interpolate NaN values in r_lx 
        l_ly = interpolate_nans(l_ly)  # Interpolate NaN values in l_ly
        r_ly = interpolate_nans(r_ly)  # Interpolate NaN values in r_ly
    except Exception as e:
        print(f"Error reading left eye data files: {e}")
else:
    print("Warning: Left eye data files are not present. Saccade detection might be bad especially for rats!!")

## Read the VD axis data for left eye
if (blink_detection_l==1):
    try:
        vdaxis_L_data = np.genfromtxt(clean_csv(vdaxis_L_file), delimiter=',', skip_header=1, dtype=np.float64)
        [vd_frame_l, vd_l_ts, vd_l_lx, vd_l_ly, vd_l_rx, vd_l_ry] = vdaxis_L_data[:, 0], vdaxis_L_data[:, 1], vdaxis_L_data[:, 2], vdaxis_L_data[:, 3], vdaxis_L_data[:, 4], vdaxis_L_data[:, 5]
        vd_frame_l = vd_frame_l.astype(int)

        # Interpolate NaN values
        vd_l_lx = interpolate_nans(vd_l_lx)
        vd_l_ly = interpolate_nans(vd_l_ly)
        vd_l_rx = interpolate_nans(vd_l_rx)
        vd_l_ry = interpolate_nans(vd_l_ry)
    except Exception as e:
        print(f"Error reading left VD axis data file: {e}") 

        
### Read the IMU data for the accelerometer and gyroscope
imu_data = np.genfromtxt(IMU_file, delimiter=',', skip_header=1, dtype=np.float64)
[imu_time,a_x,a_y,a_z,g_x,g_y,g_z,m_x,m_y,m_z] = imu_data[:, 0], imu_data[:, 1], imu_data[:, 2], imu_data[:, 3], imu_data[:, 4], imu_data[:, 5], imu_data[:, 6], imu_data[:, 7], imu_data[:, 8], imu_data[:, 9]
imu_time = imu_time.astype(np.float64)  # Ensure imu_time is in float64 format
# Interpolate NaN values in IMU data
a_x = interpolate_nans(a_x)
a_y = interpolate_nans(a_y)
a_z = interpolate_nans(a_z)
g_x = interpolate_nans(g_x)
g_y = interpolate_nans(g_y)
g_z = interpolate_nans(g_z)
m_x = interpolate_nans(m_x)
m_y = interpolate_nans(m_y)
m_z = interpolate_nans(m_z)


Error reading left VD axis data file: expected str, bytes or os.PathLike object, not NoneType


## Sanity check: How far apart are the stimuli?  


In [50]:

d_frames = np.diff(go_frame)    # successive differences (frames)
d_sec = d_frames / ttl_freq

plt.figure(figsize=(8,3))
plt.plot(d_sec, marker='o')
plt.xlabel('Stimulus index')
plt.ylabel('Δtime (s) to next stim')
plt.title('Seconds between successive Go Stims')
plt.grid(alpha=.3)
plt.tight_layout()
plt.show()


## Analyze and plot saccade statistics

In [ ]:
# --------------------------------------------------------------
# Decide which direction arrays we have
if new_go_data_format:              # recent datasets (have X & Y dirs)
    go_x = go_direction_x           # LR codes  (Left <0, Right >0)
    go_y = go_direction_y           # UD codes  (Down <0, Up >0)
else:                               # legacy datasets: only one array
    if stim_type == "UD":
        go_x = None                 # no LR info
        go_y = go_direction         # use single array as UD codes
    else:                           # "LR" or "None"
        go_x = go_direction         # use single array as LR codes
        go_y = None

# --------------------------------------------------------------
# ONE analyse-&-plot call for the right eye
right = analyze_eye_saccades(
    l_rx, l_ry, r_rx, r_ry,
    eye_rx, eye_ry,
    eye_frame_r,
    cal,
    go_frame,
    go_dir_x   = go_x,            # set earlier
    go_dir_y   = go_y,
    stim_type  = stim_type,       # "LR", "UD", "Interleaved", "None"
    blink_detection = blink_detection_r,
    vd_axis_lx = vd_r_lx, vd_axis_ly = vd_r_ly,
    vd_axis_rx = vd_r_rx, vd_axis_ry = vd_r_ry,
    saccade_threshold       = saccade_thresh,
    blink_velocity_threshold= blink_thresh
)

plot_eye_saccades(
    right["eye_camera"],
    right["eye_vel"],
    right["saccade_indices"],
    right["saccade_frames"],
    right["stim_frames"],
    saccade_window= saccade_win*ttl_freq,
    session_path = folder_path,
    stim_type    = stim_type,
    eye_name     = "Right Eye",
)


In [ ]:
# eye_camera = right["eye_camera"]
# eye_camera_diff=    right["eye_vel"]
# saccade_indices=    right["saccade_indices"]
# saccade_frames=    right["saccade_frames"]
# stim_frames=    right["stim_frames"]
# saccade_window= saccade_win*ttl_freq
# session_path = folder_path
# stim_type    = stim_type
# eye_name     = "Right Eye"

# print(len(saccade_indices), len(saccade_frames))

# session_name = os.path.basename(session_path.rstrip("/\\"))

# # ───────── global axis limits (all saccades) ─────────
# x_all = eye_camera[saccade_indices, 0]
# y_all = eye_camera[saccade_indices, 1]
# pad   = 0.10
# rngX  = x_all.max() - x_all.min()
# rngY  = y_all.max() - y_all.min()
# X_LIM = (x_all.min() - pad*rngX, x_all.max() + pad*rngX)
# Y_LIM = (y_all.min() - pad*rngY, y_all.max() + pad*rngY)

# max_abs = np.max(np.abs(eye_camera_diff))
# angle_all = np.arctan2(eye_camera_diff[saccade_indices, 1],
#                         eye_camera_diff[saccade_indices, 0])
# n_all = len(saccade_indices)

# # ───────── master figure (ALL saccades) ─────────
# fig = plt.figure(figsize=(10, 6))
# gs  = gridspec.GridSpec(2, 2, width_ratios=[3, 2])
# ax_quiver = fig.add_subplot(gs[:, 0])
# ax_polar  = fig.add_subplot(gs[0, 1], polar=True)
# ax_linear = fig.add_subplot(gs[1, 1])

# ax_quiver.set_xlim(*X_LIM); ax_quiver.set_ylim(*Y_LIM)
# ax_quiver.set_xlabel('X (°)'); ax_quiver.set_ylabel('Y (°)')
# ax_quiver.set_title(
#     f"{session_name}\nAll saccades ({n_all}) — {eye_name}  (stim: {stim_type})"
# )

# cols = np.array([vector_to_rgb(a, max_abs) for a in angle_all])
# ax_quiver.quiver(x_all, y_all,
#                     eye_camera_diff[saccade_indices, 0],
#                     eye_camera_diff[saccade_indices, 1],
#                     angles='xy', scale_units='xy', scale=1,
#                     color=cols, alpha=.5)

# # PCA arrows (unchanged)
# pca.fit(eye_camera_diff[saccade_indices] /
#         np.linalg.norm(eye_camera_diff[saccade_indices], axis=1, keepdims=True))
# for i, (vec, var) in enumerate(zip(pca.components_, pca.explained_variance_ratio_)):
#     ax_quiver.arrow(np.mean(x_all), np.mean(y_all),
#                     *(vec * 10 * np.sqrt(var)),
#                     color=['k', 'b'][i], width=0.1,
#                     label=f'PC{i+1} ({var:.2f} var)')
# ax_quiver.legend()

# plot_angle_distribution(angle_all, ax_polar)
# plot_linear_histogram(angle_all, ax_linear)

# # save master figure
# all_fname = f"{session_name}_{eye_name}_ALL_{stim_type}.png"
# fig.savefig(results_dir / all_fname, dpi=300, bbox_inches='tight')


# # Determine the overall frame range [0, last_frame]
# last_frame = int(saccade_frames.max())
# clipped_any = False
# plot_window = np.arange(0,saccade_window,1)

# # ───────── one figure per stimulus label (skip "All") ─────────
# for label, frames in stim_frames.items():
#     if label == "All":
#         continue

#     # gather saccades within ±plot_window around each stim

#     idx_buf = []  # buffer to collect saccade indices for this label

#     for f in frames:
#         lower_bound = max(f + plot_window[0], 0)
#         upper_bound = min(f + plot_window[-1], saccade_frames.max())

#         for sf, idx in zip(saccade_frames, saccade_indices):
#             if lower_bound <= sf <= upper_bound:
#                 idx_buf.append(idx)


#     idx_use = np.array(idx_buf, dtype=int)
#     if idx_use.size == 0:
#         continue

#     ang = np.arctan2(eye_camera_diff[idx_use, 1],
#                         eye_camera_diff[idx_use, 0])
#     n_cond = len(idx_use)

#     fig = plt.figure(figsize=(6, 3))
#     gs  = gridspec.GridSpec(2, 2, width_ratios=[3, 2])
#     ax_q = fig.add_subplot(gs[:, 0])
#     ax_p = fig.add_subplot(gs[0, 1], polar=True)
#     ax_l = fig.add_subplot(gs[1, 1])

#     ax_q.set_xlim(*X_LIM); ax_q.set_ylim(*Y_LIM)
#     ax_q.set_xlabel('X (°)'); ax_q.set_ylabel('Y (°)')
#     ax_q.set_title(f"{session_name}\n{eye_name} — {label} (n={n_cond})")

#     cols = np.array([vector_to_rgb(a, max_abs) for a in ang])
#     ax_q.quiver(eye_camera[idx_use, 0], eye_camera[idx_use, 1],
#                 eye_camera_diff[idx_use, 0], eye_camera_diff[idx_use, 1],
#                 angles='xy', scale_units='xy', scale=1,
#                 color=cols, alpha=.5)

#     plot_angle_distribution(ang, ax_p)
#     plot_linear_histogram(ang, ax_l)

#     fig.tight_layout()
#     fname = f"{session_name}_{eye_name}_{label.replace('/','-')}.png"
#     fig.savefig(results_dir / fname, dpi=300, bbox_inches='tight')

677 677


## Look at probability of saccades as a function of stimulus onset

In [53]:

fig = plot_saccade_probability(
    go_frame        = go_frame,
    go_dir_x        = go_x,      # or None if not recorded
    go_dir_y        = go_y,      # or None
    saccade_frames  = right["saccade_frames"],
    ttl_freq        = ttl_freq,
    t_window_s      = 0.5,
    bin_ms          = 50
)

session_name = os.path.basename(folder_path.rstrip("/\\"))
eye_name = 'Right Eye'  # or 'Left Eye' if you analyze the left eye

# optional: save alongside other figures
prob_fname = f"{session_name}_{eye_name}_StimLockedProb.png"
fig.savefig(results_dir / prob_fname, dpi=300, bbox_inches='tight')

## Sanity check: Plot an overlay of Go-stims and Saccades 

In [54]:
# ============================================================
# Timeline plot: saccades (grey) + colour-coded stimuli
# ============================================================
# -----------------------------------------------------------------
# Assumes these objects already exist in the workspace
# -----------------------------------------------------------------
# right["saccade_frames"]   – Bonsai frame IDs of all saccades
# go_frame                  – Bonsai frame IDs of stimuli
# go_dir_x, go_dir_y        – direction codes (can be None)
# ttl_freq                  – camera TTL rate (Hz)
# results_dir               – Path(folder_path) / "Results"
# -----------------------------------------------------------------

# 1) prep data ----------------------------------------------------
saccade_frames = np.asarray(right["saccade_frames"], dtype=int)

# fallback arrays if dir arrays are missing
gx = go_x if (go_x is not None) else np.zeros_like(go_frame)
gy = go_y if (go_y is not None) else np.zeros_like(go_frame)


# ── 1-bis. count how many of each stimulus -----------------------------
eps = 1e-6                      # tolerance for “zero”
is_left   = (np.abs(gy) < eps) & (gx < -eps)
is_right  = (np.abs(gy) < eps) & (gx >  eps)
is_down   = (np.abs(gx) < eps) & (gy < -eps)
is_up     = (np.abs(gx) < eps) & (gy >  eps)

n_left, n_right = is_left.sum(),  is_right.sum()
n_down, n_up    = is_down.sum(),  is_up.sum()


# palette mapping
palette = {'L': 'green', 'R': 'pink', 'D': 'blue', 'U': 'red', 'NA': 'gray'}

# build colour list per stimulus
colors = []
for x, y in zip(gx, gy):
    if abs(y) > 1e-6:                       # Up / Down has priority
        colors.append(palette['U' if y > 0 else 'D'])
    elif abs(x) > 1e-6:                     # Left / Right
        colors.append(palette['R' if x > 0 else 'L'])
    else:
        colors.append(palette['NA'])

# sort frames & colours together
order      = np.argsort(go_frame)
t_stim     = go_frame[order] / ttl_freq
colors     = [colors[i] for i in order]

# convert saccade frames to seconds
t_sacc = np.sort(saccade_frames) / ttl_freq

# 2) plot ---------------------------------------------------------
fig, ax = plt.subplots(figsize=(12, 2.5))

# saccades: grey vertical ticks at y = 0
ax.vlines(t_sacc, -0.1, 0.1, colors='0.25', linewidth=1)

# stimuli: colour ticks at y = 1
ax.vlines(t_stim, 0.9, 1.1, colors=colors, linewidth=2)

# axes formatting
ax.set_yticks([0, 1])
ax.set_yticklabels(['Saccade', 'Stim'])
ax.set_xlabel('Time (s)')
ax.set_title('Timeline of saccades and stimuli')
ax.set_xlim(t_sacc.min() - 1, t_sacc.max() + 1)
ax.set_ylim(-0.5, 1.5)
ax.grid(axis='x', alpha=.3)

# legend
handles = [
    mlines.Line2D([], [], color='0.25', marker='|', ls='', markersize=10,
                  label='Saccade'),
    mlines.Line2D([], [], color='green', marker='|', ls='', markersize=10,
                  label=f'Stim Left  (n={n_left})'),
    mlines.Line2D([], [], color='pink',  marker='|', ls='', markersize=10,
                  label=f'Stim Right (n={n_right})'),
    mlines.Line2D([], [], color='blue',  marker='|', ls='', markersize=10,
                  label=f'Stim Down  (n={n_down})'),
    mlines.Line2D([], [], color='red',   marker='|', ls='', markersize=10,
                  label=f'Stim Up    (n={n_up})')
]
ax.legend(handles=handles, loc='upper right', ncol=5, fontsize=9, framealpha=.9)

plt.tight_layout()

# 3) save ---------------------------------------------------------
results_dir = Path(results_dir)            # ensure Path type
results_dir.mkdir(exist_ok=True)
out = results_dir / "timeline_saccade_vs_stim.png"
fig.savefig(out, dpi=300, bbox_inches='tight')
print("Saved timeline plot →", out)


Saved timeline plot → X:\Experimental_Data\EyeHeadCoupling_RatTS_server\TSh01_Paris_server\Tsh001_2025-06-11T13_02_29\Results\timeline_saccade_vs_stim.png


## Plot how many stims produced at least one saccade

In [55]:
# ============================================================
#  Probability of ≥1 saccade within 0.5 s of each stimulus


# ----- parameters -------------------------------------------
win     = 0.5                   # seconds after onset
w_frames = int(win * ttl_freq)  # convert to frames

# direction masks
gx = go_x if go_x is not None else np.zeros_like(go_frame)
gy = go_y if go_y is not None else np.zeros_like(go_frame)

dir_info = {
    'Left' :  (gx < -1e-6,  'green'),
    'Right':  (gx >  1e-6,  'pink'),
    'Down' :  (gy < -1e-6,  'blue'),
    'Up'   :  (gy >  1e-6,  'red')
}

labels, probs, colors = [], [], []

for label, (mask, col) in dir_info.items():
    stim_frames = go_frame[mask]
    n_stim      = len(stim_frames)
    if n_stim == 0:
        continue                                 # skip if this direction absent
    # check each stimulus: does ANY saccade happen within +win seconds?
    has_sacc = [( (saccade_frames >= f) & (saccade_frames <= f + w_frames) ).any()
                for f in stim_frames]
    prob = np.mean(has_sacc)                    # fraction of stimuli with ≥1 sac
    labels.append(label)
    probs.append(prob)
    colors.append(col)
    print(f"{label:5s}: {prob*100:5.1f}%  ({sum(has_sacc)}/{n_stim} stimuli)")

# ----- bar chart --------------------------------------------
fig, ax = plt.subplots(figsize=(6,4))
ax.bar(labels, probs, color=colors, edgecolor='k')
ax.set_ylim(0, 1)
ax.set_ylabel(f"P(saccade within {win}s)")
ax.set_title('Probability of a saccade in first 0.5 s after stimulus')
ax.grid(axis='y', alpha=.3)
plt.tight_layout()

# ----- save (optional) --------------------------------------
out = results_dir / f"saccade_prob_within_{win*1000:.0f}ms.png"
fig.savefig(out, dpi=300, bbox_inches='tight')
print("Saved probability bar-plot →", out)

Left :  45.1%  (23/51 stimuli)
Right:  38.1%  (16/42 stimuli)
Down :  42.1%  (16/38 stimuli)
Up   :  54.0%  (27/50 stimuli)
Saved probability bar-plot → X:\Experimental_Data\EyeHeadCoupling_RatTS_server\TSh01_Paris_server\Tsh001_2025-06-11T13_02_29\Results\saccade_prob_within_500ms.png
